In [11]:
import torch
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

In [12]:
data_transforms = transforms.Compose([transforms.Resize((128, 128)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean = [0.485, 0.456, 0.406],
                                                          std = [0.229, 0.224, 0.225])
                                     ])

dataset = ImageFolder(root = "./Dataset", transform=data_transforms)


In [14]:

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

In [16]:
import torch.nn as nn
import torch.nn.functional as F

class WeatherCNN(nn.Module):

    def __init__(self):
        super(WeatherCNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.fc1 = nn.Linear(128 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, 4)
    
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = WeatherCNN()
        

In [20]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)


num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        optimizer.zero_grad() # setting gradients as zero
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')


    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Validation Loss: {val_loss/len(val_loader):.4f}, Accuracy: {100 * correct/total:.2f}%')



Epoch [1/10], Loss: 0.4800
Validation Loss: 0.2547, Accuracy: 88.44%
Epoch [2/10], Loss: 0.3913
Validation Loss: 0.2687, Accuracy: 89.78%
Epoch [3/10], Loss: 0.2884
Validation Loss: 0.2423, Accuracy: 88.00%
Epoch [4/10], Loss: 0.2128
Validation Loss: 0.1990, Accuracy: 92.44%
Epoch [5/10], Loss: 0.1172
Validation Loss: 0.1681, Accuracy: 92.89%
Epoch [6/10], Loss: 0.0621
Validation Loss: 0.2270, Accuracy: 93.33%
Epoch [7/10], Loss: 0.0730
Validation Loss: 0.2494, Accuracy: 92.44%
Epoch [8/10], Loss: 0.0330
Validation Loss: 0.1599, Accuracy: 93.78%
Epoch [9/10], Loss: 0.0247
Validation Loss: 1.6280, Accuracy: 93.78%
Epoch [10/10], Loss: 0.0449
Validation Loss: 0.3310, Accuracy: 90.22%
